<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
EN to ZN
1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer
2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning
3. Need a tokenizer that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)
4. Fine-tune, may need prompt + EN,CN pair.
5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)
6. Don't need mask in decoder only model
7. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension) so if the intput lenghth is 10 runs (assuming we have 10 input length, prompt (EN) + ZH tokens + newly generated EN tokens),
   and the maximum length is 20, we would go through the decoder model iteratively 10 more times or until meets the EOS token.
8. Each layer of transformer would pass down output shape (batch, seq, embedding dimension), and we can pick up the last token and get   to know how the representation evolved.
9. For the intermediate weights of FFN of each layers of transformer, it has representation shape of  (,768) and we can also put it into the comparison to the last token of the final transformer layer.

"""

"\nEN to ZN\n1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer\n2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning\n3. Need a tokenizer that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)\n4. Fine-tune, may need prompt + EN,CN pair.\n5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)\n6. Don't need mask in decoder only model\n7. No CLS token in decoder based model, conventionally, we pick up the last token of each output Nth (, embedding dimension) so if the intput lenghth is 10 runs (assuming we have 10 input length, prompt (EN) + ZH tokens + newly generated EN tokens),\n   and the maximum length is 20, we would go through the decoder model iteratively 10 more times or until meets the EOS token.\n8. Each layer of transformer would p

Package installation and importation

In [1]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output
from transformers import BertTokenizer, GPT2LMHeadModel
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from transformers import TFGPT2LMHeadModel, GPT2Config
import tensorflow as tf

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-be8d1a74-fe2b-995a-7b82-89a6fe16c5db)


In [4]:
#check out the data source we have
tmp_builder = tfds.builder("wmt19_translate/zh-en")
pprint(tmp_builder.subsets)

{Split('train'): ['newscommentary_v14',
                  'wikititles_v1',
                  'uncorpus_v1',
                  'casia2015',
                  'casict2011',
                  'casict2015',
                  'datum2015',
                  'datum2017',
                  'neu2017'],
 Split('validation'): ['newstest2018']}


In [5]:
#download data by tfds.builder
config = tfds.translate.wmt.WmtConfig(
  version=tfds.core.Version('0.0.3'),
  language_pair=("zh", "en"),
  subsets={
    tfds.Split.TRAIN: ["newscommentary_v14"]
  }
)
builder = tfds.builder("wmt_translate", config=config)

# Download and prepare the dataset
builder.download_and_prepare()

# Set builder to dataset (data pipeline type), split it into training, validation, testing
examples = builder.as_dataset(
    split=['train[:10%]', 'train[10%:11%]', 'train[11%:12%]'],
    as_supervised=True
)

train_data, val_data, test_data = examples

train_examples, train_labels = next(iter(train_data.batch(5000)))
val_examples, val_labels = next(iter(val_data.batch(500)))
test_examples, test_labels = next(iter(test_data.batch(10)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3.incomplete2GVLX4/wmt_translate-train.tfrecord*..…

Dataset wmt_translate downloaded and prepared to /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3. Subsequent calls will reuse this data.


In [6]:
train_examples[:4]

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'The fear is real and visceral, and politicians ignore it at their peril.',
       b'In fact, the German political landscape needs nothing more than a truly liberal party, in the US sense of the word \xe2\x80\x9cliberal\xe2\x80\x9d \xe2\x80\x93 a champion of the cause of individual freedom.',
       b'Shifting to renewable-energy sources will require enormous effort and major infrastructure investment.',
       b'In this sense, it is critical to recognize the fundamental difference between \xe2\x80\x9curban villages\xe2\x80\x9d and their rural counterparts.'],
      dtype=object)>

In [7]:
#set up the input as prompt + EN and target label as ZH
#tokenized it using the tokenizer that can both handle EN and ZH

train_examples_str = [x.decode('utf-8') for x in train_examples.numpy()]
train_labels_str = [x.decode('utf-8') for x in train_labels.numpy()]
val_examples_str = [x.decode('utf-8') for x in val_examples.numpy()]
val_labels_str = [x.decode('utf-8') for x in val_labels.numpy()]
test_examples_str = [x.decode('utf-8') for x in test_examples.numpy()]
test_labels_str = [x.decode('utf-8') for x in test_labels.numpy()]



# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_finetuned = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)
tokenizer.pad_token = tokenizer.eos_token

# Add a prompt to each input example
prompt = "Translate English to Chinese: "

input_texts_train = [prompt + text for text in train_examples_str]
# Tokenize inputs and labels
input_train_tokenized = tokenizer(input_texts_train, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
label_train_tokenized = tokenizer(train_labels_str, return_tensors='tf', padding='max_length', truncation=True, max_length=100)


input_texts_val = [prompt + text for text in val_examples_str]
# Tokenize inputs and labels
input_val_tokenized = tokenizer(input_texts_val, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
label_val_tokenized = tokenizer(val_labels_str, return_tensors='tf', padding='max_length', truncation=True, max_length=100)

input_texts_test = [prompt + text for text in test_examples_str]
# Tokenize inputs and labels
input_test_tokenized = tokenizer(input_texts_test, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
label_test_tokenized = tokenizer(test_labels_str, return_tensors='tf', padding='max_length', truncation=True, max_length=100)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [8]:
# For training data
gpt_train_inputs = {
    'input_ids': input_train_tokenized['input_ids'],
    'attention_mask': input_train_tokenized['attention_mask']
}

# For validation data
gpt_val_inputs = {
    'input_ids': input_val_tokenized['input_ids'],
    'attention_mask': input_val_tokenized['attention_mask']
}

In [9]:
def gpt_loss(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # Masking to avoid calculating loss on padding tokens
    mask = tf.cast(tf.not_equal(labels, 0), tf.float32)
    loss = loss_fn(labels, logits, sample_weight=mask)
    return loss

model_finetuned.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
                        loss=gpt_loss)

history = model_finetuned.fit(gpt_train_inputs,
                              label_train_tokenized['input_ids'],
                              epochs=2,
                              batch_size=8,
                              validation_data=(gpt_val_inputs, label_val_tokenized['input_ids']))

Epoch 1/2
625/625 [==============================] - 531s 732ms/step - loss: 3.9403 - val_loss: 3.6900
Epoch 2/2
625/625 [==============================] - 460s 736ms/step - loss: 3.7680 - val_loss: 3.6567


In [10]:
test_examples_str[:4]

['Beyond showing no benefit from mammography screening, the Canadian trial highlighted one of the practice’s major disadvantages: over-diagnosis.',
 '(These and related issues were raised at a recent event on Capitol Hill in which I took part.)',
 'The world economy needs them – and no consumer-facing company can afford to ignore them.',
 'It used to be difficult to find anything to do on a Saturday night in Shanghai.']

In [47]:
test_labels_str[:4]

['加拿大试验不仅未能证明乳腺X光筛查的好处，还突出了上述方法的一个主要缺陷：诊断过度。',
 '（这些问题以及相关问题在最近的国会山会议上被提出，我也参与了这次会议。 ）',
 '世界经济需要他们 — — 任何面向消费者的公司可以忽视他们。',
 '以前在上海，星期六晚上无事可做。']

In [44]:
input_text = prompt + test_examples_str[0]
tokenized_input = tokenizer.encode(input_text, return_tensors='tf', padding='max_length', truncation=True, max_length=100)  # Reduced max_length for input

# # Check the actual length of the input and adjust max_length accordingly
# actual_length_of_input = tf.math.count_nonzero(tokenized_input, axis=-1).numpy()[0]
# new_max_length = actual_length_of_input + 20  # Allow room for generation


output_ids = model_finetuned.generate(tokenized_input, max_length=120, num_return_sequences=1)
translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
translation

'Translate English to Chinese: Beyond showing no benefit from mammography screening, the Canadian trial highlighted one of the practice’s major disadvantages: over-diagnosis.��������������������'

In [45]:
output_ids[0]

<tf.Tensor: shape=(120,), dtype=int32, numpy=
array([ 8291, 17660,  3594,   284,  3999,    25, 12197,  4478,   645,
        4414,   422, 13418,  4867, 14135,    11,   262,  5398,  4473,
       14537,   530,   286,   262,  3357,   447,   247,    82,  1688,
       38457,    25,   625,    12, 47356,  5958,    13, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256,   120,   120,   120,   120,   120,   120,   120,   120,
         120,   120,   120,   120,   120,   120,   120,   120,   120,
         120,   120,   120], dtype=int32)>

In [46]:
input_text

'Translate English to Chinese: Beyond showing no benefit from mammography screening, the Canadian trial highlighted one of the practice’s major disadvantages: over-diagnosis.'

In [57]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2-medium", pad_token_id=tokenizer.eos_token_id)
tokenizer.pad_token = tokenizer.eos_token

In [59]:
def predict_sentiment(model, tokenizer, review_text):
    input_text = "Sentiment Analysis: " + review_text
    tokenized_input = tokenizer.encode(input_text, return_tensors='tf', padding='max_length', truncation=True, max_length=100)
    output_ids = model.generate(tokenized_input, max_length=256, num_return_sequences=1)
    prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return prediction

# Example usage
review = "This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors."
sentiment_prediction = predict_sentiment(model, tokenizer, review)
print(sentiment_prediction)

Sentiment Analysis: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors.The first----------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
#GPT2 model can't handle EN to ZH tasks even after fine-tuning. It may work but it need a lots of data set which is not feasible in this stage
#It's bad at sentiment analysis as well, can we fine tune it and make it able to do the task?